# Item 11: Use zip to Process Iterators in Parallel

- Often in Python you find yourself with many lists of related objects. List comprehensions make it easy to take a source list and get a derived list by applying an expression.

In [1]:
names = ['Cecilia', 'Lise', 'Marie']
letters = [len(n) for n in names]

- The items in the dericed list are related to the items in the source list by their indexes. To iterate over both lists in parallel, you can iterate over the length of the names source list.

In [4]:
longest_name = None
max_letters = 0

for i in range(len(names)):
    count = letters[i]
    if count > max_letters:
        longest_name = names[i]
        max_letters = count
        
print(longest_name)

Cecilia


- To make this code clearer, Python provides the zip built-in function. In Python 3, zip wraps two or more iterators with a lazy generator. The zip generator yields tuples. containing the next value from each iterator. The resulting code is mach clearer than indexing into multiple lists.

In [5]:
for name, count in zip(names, letters):
    if count > max_letters:
        longest_name = name
        max_letters = count

In [7]:

print(longest_name)

Cecilia


- There are two problems with the zip built-in.
- The first issue is that in Python 2 zip is not a generator; it will fully exhaust the supplied iterators and return a list of all the tuples it creates. This could potentially use a lot of memory and cause your program to crash. If you want to zip very large iterators in Python 2, you should use izip from the itertools built-in module.

- The second issue is that zip behaves stragely if the input iterators are of different lengths. For example, say you add anoter name to the list above but forget to update the letter counts. Running zip on the two input lists will have an unexpected result.

In [8]:
names.append('Rosalind')
for name, count in zip(names, letters):
    print(name)

Cecilia
Lise
Marie


- The new item for 'Rosalind' isn't there. This it just how zip works. It keeps yeilding tuples until a wrapped iterator is exhausted. This approach works fine when you know that the iterators are of the same length, which is often the case for derived lists created by list comprehensions. In many other cases, the truncating behavior of zip is surprising and bad. If you aren't confident that the lengths of the lists you want to zip equal, condifer using the zip_longest function from the itertools built-in module instead.

## Things to Remember

- The zip built-in function can be used to iterate over multiple iterators in parallel.
- In Python 3, zip is a lazy generator that produces tuples. In Python 2, zip returns the full result as a list of tuples.
- zip truncates its output silently if you supply it with iterators of different lengths.
- The zip_longest function from the itertools built-in module lets you iterate over multiple iterators in parallel regardless of their lengths.

# Item 12: Avoid else Blocks After for and while Loops

- Python loops have an extra feature that is not available in most other programming languages: you can put an else block immediately after a loop's repeated interior block.

In [9]:
for i in range(3):
    print('Loop %d' % i)
else:
    print('Else block!')

Loop 0
Loop 1
Loop 2
Else block!


- Surprisingle, the else block runs immediately after the loop finishes. Why is the clause called 'else'? Why not 'and'? In an if/else statement, else means, 'Do  this if the block before this doesn't happen.' In a try/except statement, except has the same difinition: 'Do this if trying the block before this failed.'


- Similarly, else from try/except/else follows this pattern because it means, 'Do this if the block before did not fail.' try/finally is also intuitive because it means, 'Always do what is final after trying the block before.'

- Given all of the uses of else, except, and finally in Python, a new programmer might assume that the else part of for/else means, 'Do this if the loop wasn't completed.' In reality, it does exactly the opposite. Using a break statement in a loop will actually skip the else block.

In [10]:
for i in range(3):
    print('Loop %d' % i)
    if i == 1:
        break
else:
    print('Else block!')

Loop 0
Loop 1


- Another surprise is that the else block will run immediately if you loop over an empty squence.

In [11]:
for x in []:
    print('Never runs')
else:
    print('For Else block!')

For Else block!


- The rationale for these behaviors is that else blocks after loops are useful when you're using loops to search for something. For example, say you want to determine whether two numbers are coprime (their only common divisor is 1). Here, I iterate through every possible common divisor and test the numbers. After every option has been tried, the loop ends. The else block runs when the numbers are coprime because the loop doesn't encounter a break.

In [12]:
a = 4
b = 9
for i in range(2, min(a, b) + 1):
    print('Testing', i)
    if a % i == 0 and b % i == 0:
        print('Not coprime')
        break
else:
    print('Coprime')

Testing 2
Testing 3
Testing 4
Coprime


- in pratice, you wouldn't write the code this way. Instead, you'd write a helper function to do the calculation. Such a helper function is written in two common style. 
- The first approach is to return early when you fund the coindition you're looking for. You return the default outcome if you fall through the loop.

In [13]:
def ciprime(a, b):
    for i in range(2, mun(a, b)+1):
        if a % i == 0 and b % i == 0:
            return False
    return True


In [14]:
def coprime2(a, b):
    is_coprime = True
    for i in range(2, min(a, b) + 1):
        if a % i == 0 and b % i == 0:
            is_coprime = False
            break
    return is_coprime

- Both of theses approaches are much clearer to readers of unfamiliar code. The expressivity you gain from the else block doesn't outweigh the burden you put on people who want to understand your code in the future. Simple constructs like loops should be self-evident in Python. You should avoid using else blocks after loops entirely.

## Things to Remember

- Python has special syntax that allows else blocks to immediately follow for and while loop interior blocks.
- The else block after a loop only runs if the loop body did not encounter a break statement.

- Avoid using else blocks after loops because their behavior isn't intuitive and can be confusing.

# Item 13: Take Advantage of Each Block in try/except/else/finally

- There are four distinct times that you may want to take action during exception handling in Python. These are captured in the functionality of try, except, and finally blocks. Each block serves a unique purpose in the compound statement, and their various combinations are useful.

### Finally Blocks

- One common usage of try/finally is for reliably closing file handles.

In [ ]:
handle = open('/tmp/random_data.txt')
try:
    data = handle.read()
finally:
    handle.close()

- Any exception raised by the read method will always propagate up to the calling code, yet the close method of handle is also guaranteed to run in the finally block. You must call open before the try block because exceptions that occur when openning the file should skip the finally block.

### Else Blocks

- Use try/except/else to make it clear which exceptiongs will be handled by your code and which exceptions will propagate up. When the try block doesn't raise an exception, the else block will run. The else block helps you minimize the amount of code in the try block and improves readability. For example, say you want to load JSON dictionary data from a string and return the value of a key it contains.

In [15]:
def load_json_key(data, key):
    try:
        result_dict = json.loads(data)
    except ValueError as e:
        raise KeyError from e
    else:
        return result_dict[key]
    

- If the data isn't valid JSON, then decoding with json.loads will raise a ValueError. The exception is caught by the except block and handled. If decoding is successful, then the key lookup will occur in the else block. If the key lookup raises any exceptions, they will propagate up to the caller because they are outside the try block. The else clause ensures that what follows the try/except is visually distinguished from the excecpt block. This makes the exception propagation behavior clear.

### Everything Together

- Use try/except/else/finally when you want to do it all in one compound statement. For example, say you want to read a description of work to do from a file, process it, and then update the file in place. Here, the try block is used to read the file and process it. The except block is used to handle exceptions from the try block that are expected. The else block is used to update the file in place and to allow related exceptions to propagate up. The finally block cleans up the file handle.

In [ ]:
UNDEFINED = object()

def divide_json(path):
    handle = open(path, 'r+')
    try:
        data = handle.read()
        op = json.loads(data)
        value = (
            op['nemerator'] /
            op['denominator'])
        except ZeroDivisionError as e:
            return UNDEFINED
        else:
            op['result'] = value
            result = json.dumps(op)
            handle.seek(0)
            handle.write(result)
            return value
        finally:
            handle.close()
            

- This layout is especially useful because all of the blocks work together in intuitive ways. For example, if an exception gets raised in the else block while rewriting the result data, the finally block will still run and close the file handle.

### Things to Remember

- The try/finally compound statement lets you run cleanup code regardless of whether exceptions were raised in the try block.
- The else block helps you minimize the amount of code in try blocks and visually distinguish the success case from the try/except blocks.
- An else block can be used to perfrom additional actions after a successful try block but before common cleanup in a finally block.

# 2. Functions

- The first organizational tool programmers use in Python is the function. As in other programming languages, functions enable you to break large programs into smaller, simpler pieces. They improve readability and make code more approachable. They allow for reuse and refactoring.

- Functions in Python have a variety of extra features that make the pregrammer's life easier. Some are similar to capabilities in other programming languages, but many are unique to Python. These extras can make a function's purpose more obvious. They can eliminate noise and clarify the intention of callers. They can significantly reduce subtle bugs that are difficult to find.

# Item 14: Prefer Exceptions to Returning None

- When writing utility functions, there's a draw for Python programmers to give special meaning to the return value of None. It seems to makes sense in some cases. For example, say you want a helper function that divides one number by another. In the case of dividing by zero, returning None seems natural because the result is undefined.

In [16]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

- What happens when the numerator is zero? That will cause the return value to also be zero. This can cause problems when you evaluate the result in a condition like an if statement. You may accidentally look for any False equivalent value to indicate errors instead of only looking for None.

In [ ]:
x, y = 0, 5
result = devide(x, y)
if not result:
    print('Invalid inputs')

- This is a common mistake in Python code when None has special meaning. This is why returning None from a function is error prone. There are two ways to reduce the change of such errors.

- The first way is to split the return value into a two-tuple. The first part of the tuple indicates that the operation was a success or failure. The second part  is the actual result that was computed.

In [17]:
def divide(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None